In [1]:
! pip install --upgrade praw

  Using cached websocket_client-1.8.0-py3-none-any.whl.metadata (8.0 kB)
Using cached websocket_client-1.8.0-py3-none-any.whl (58 kB)



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: C:\Users\sreer\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import praw
import datetime
import pandas as pd

In [3]:
reddit = praw.Reddit(
    client_id='fWTdhJzk8f_L6KidOU4Y8w',
    client_secret='2JvZ9ZkiPWGJlfhngBZPoxThlxpa4g',
    user_agent='mydscstudy/1.0 by Yeageristeren17'
)

# Subreddit verification
data_science_sr = reddit.subreddit('datascience')
print(f"Subreddit Title: {data_science_sr.display_name}")

Subreddit Title: datascience


In [4]:
rposts = []

start = datetime.datetime(2023,1,1).timestamp()
stop = datetime.datetime.now().timestamp()

min_score = 50
title_words = 5
post_count = 100
comments_per_post = 3

In [5]:
def timestamp_to_date(timestamp):
    return datetime.datetime.utcfromtimestamp(timestamp).strftime('%Y-%m-%d')


# Filtering posts 
for reddit_posts in data_science_sr.top(limit=None):
    if ( reddit_posts.created_utc >= start and reddit_posts.created_utc <= stop and
         reddit_posts.score > min_score and 
         len(reddit_posts.title.split()) > title_words):
        post_info = { 'Date': timestamp_to_date(reddit_posts.created_utc),'Post Score': reddit_posts.score,'Post Title': reddit_posts.title }
        
        reddit_posts.comments.replace_more(limit=None)
        comments = []
        for i, comment in enumerate(reddit_posts.comments.list()):
            if i < comments_per_post and comment.body.strip() != '[deleted]':
                comments.append(comment.body)
        if len(comments) >= comments_per_post:
            for i, comment in enumerate(comments):
                post_info[f'Top comment {i+1}'] = comment

            rposts.append(post_info)
            
        if len(rposts) >= post_count:
            break

print(rposts)

[{'Date': '2024-09-02', 'Post Score': 3173, 'Post Title': 'How to avoid 1/2-assed data analysis', 'Top comment 1': '1st of February', 'Top comment 2': "and then you set the cell to be a number and it's not 0,5 it's 4729024 for some reason", 'Top comment 3': 'I see you, too, excel at copy and paste jobs.'}, {'Date': '2023-01-27', 'Post Score': 2648, 'Post Title': 'As a hiring manager - this, this right here', 'Top comment 1': "He also has a PhD in mathematics so I'm sure that helped", 'Top comment 2': 'I’m assuming he had a strong mathematical/statistical background prior to taking the data science courses.', 'Top comment 3': "How seriously are personal projects taken? I'm trying to transition/move-adjacent from software engineering. Unfortunately, my current team has literally no work in this area and I haven't been able to find a internal move. I'm seeing what I can do over next 6-12 months to improve my resume when things get better"}, {'Date': '2024-12-29', 'Post Score': 2063, 'Post

In [6]:
rposts_df = pd.DataFrame(rposts)
rposts_df.columns = ['Date', 'Post Score', 'Post Title', 'Top Comment 1', 'Top Comment 2', 'Top Comment 3']
rposts_df.head()

,Date,Post Score,Post Title,Top Comment 1,Top Comment 2,Top Comment 3
0,2024-09-02,3173,How to avoid 1/2-assed data analysis,1st of February,and then you set the cell to be a number and i...,"I see you, too, excel at copy and paste jobs."
1,2023-01-27,2648,"As a hiring manager - this, this right here",He also has a PhD in mathematics so I'm sure t...,I’m assuming he had a strong mathematical/stat...,How seriously are personal projects taken? I'm...
2,2024-12-29,2063,My Data Science Manifesto from a Self Taught D...,"As a stats professor who teaches DS, your poin...",The other big thing you didn't mention is reco...,"Honestly, this is so refreshing to read and I ..."
3,2024-07-01,1816,"You're not helping, Excel! please STOP HELPING!!!",Why doesn't someone at Microsoft just spend so...,I have a product code like 12.30. Both tried t...,Ok this is a meme but Excel is infuriating bec...
4,2024-01-31,1705,Friendly reminder not to work too hard. You'll...,"I'm in HR Workforce management,\n\nLet me tell...",I agree. I'm going through something similar a...,"I lost my job three times in a row, so I think..."


In [7]:
# Saving data to CSV file
csv_fp = "praw_reddit_posts.csv"
rposts_df.to_csv(csv_fp, index=False)

print("Data saved to 'praw_reddit_posts.csv'")

Data saved to 'praw_reddit_posts.csv'
